<a href="https://colab.research.google.com/github/jeffheaton/app_generative_ai/blob/main/t81_559_class_10_2_streamlit_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# T81-559: Applications of Generative Artificial Intelligence
**Module 10: StreamLit**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 10 Material

Module 10: StreamLit

* Part 10.1: Running StreamLit in Google Colab [[Video]]() [[Notebook]](t81_559_class_10_1_streamlit.ipynb)
* **Part 10.2: StreamLit Introduction** [[Video]]() [[Notebook]](t81_559_class_10_2_streamlit_intro.ipynb)
* Part 10.3: Understanding Streamlit State [[Video]]() [[Notebook]](t81_559_class_10_3_streamlit_state.ipynb)
* Part 10.4: Creating a Chat Application [[Video]]() [[Notebook]](t81_559_class_10_4_chat.ipynb)
* Part 10.5: MultiModal Chat Application [[Video]]() [[Notebook]](t81_559_class_10_5_chat_multimodal.ipynb)


# Google CoLab Instructions

The following code ensures that Google CoLab is running and maps Google Drive if needed.

In [ ]:
import os

try:
    from google.colab import drive, userdata
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# OpenAI Secrets
if COLAB:
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

# Install needed libraries in CoLab
if COLAB:
    !pip install langchain langchain_openai openai streamlit

# Part 10.2: StreamLit Introduction


In this module, we will explore Streamlit, a powerful tool for creating interactive web applications with minimal code. We will go through two hands-on examples to demonstrate its capabilities:

1. Loan Amortization Table Generator: In the first example, we will use Streamlit's user interface elements to build a simple loan amortization table generator. You will see how easy it is to gather user input, process it, and display the results in an interactive format.

2. Multimodal Image Transformation: The second example showcases the power of multimodal models. We'll create an application that allows you to upload images and transforms them into cartoon versions using AI. This will give you an understanding of how Streamlit can handle image processing and display results effectively.

By the end of this module, you'll have a solid understanding of how to use Streamlit to build both data-driven and AI-enhanced applications.

## Loan Calculation Example


This code creates a loan amortization calculator using Streamlit, a library for building interactive web applications in Python. The application consists of a function to calculate the amortization schedule and a simple user interface for inputting loan details and displaying results.

The function ```calculate_amortization``` takes three parameters: ```loan_amount```, ```annual_rate```, and ```years```. It first calculates the monthly interest rate and the total number of payments over the loan's term. Using these, it computes the monthly payment using the standard loan amortization formula. It then iteratively builds an amortization schedule, calculating the interest and principal portions of each payment and updating the remaining balance. The results are stored in a list, which is then converted into a pandas ```DataFrame`` with columns for each month, payment amount, principal paid, interest paid, and remaining balance.

The Streamlit app's interface starts with a title: 'Loan Amortization Calculator'. It provides input fields for the user to specify the loan amount, annual interest rate, and loan term in years. These inputs use Streamlit's number_input method, which allows users to adjust values easily.

Upon clicking the "Calculate Amortization Table" button, the application calls the calculate_amortization function with the input values and displays the resulting DataFrame. It also displays the monthly payment amount. Additionally, the application generates a downloadable CSV file of the amortization table, allowing users to save the results locally. This functionality is provided by the st.download_button method, which takes the DataFrame's CSV data and offers it as a downloadable file.

In [ ]:
%%writefile app.py

import streamlit as st
import pandas as pd

# Function to calculate loan amortization
def calculate_amortization(loan_amount, annual_rate, years):
    monthly_rate = annual_rate / 12 / 100
    num_payments = years * 12
    monthly_payment = loan_amount * (monthly_rate * (1 + monthly_rate) ** num_payments) / ((1 + monthly_rate) ** num_payments - 1)

    # Create amortization schedule
    amortization_data = []
    balance = loan_amount
    for i in range(1, num_payments + 1):
        interest_payment = balance * monthly_rate
        principal_payment = monthly_payment - interest_payment
        balance -= principal_payment
        amortization_data.append([i, monthly_payment, principal_payment, interest_payment, max(balance, 0)])

    # Create DataFrame
    df = pd.DataFrame(amortization_data, columns=['Month', 'Payment', 'Principal', 'Interest', 'Balance'])
    return df

# Streamlit App
st.title('Loan Amortization Calculator')

# Input Fields
loan_amount = st.number_input('Loan Amount', value=300000.0, min_value=0.0, step=1000.0)
annual_rate = st.number_input('Annual Interest Rate (%)', value=7.5, min_value=0.0, step=0.1)
years = st.number_input('Loan Term (years)', value=30, min_value=1, step=1)

# Calculate amortization table
if st.button('Calculate Amortization Table'):
    amortization_df = calculate_amortization(loan_amount, annual_rate, years)
    st.write(f'Monthly Payment: ${amortization_df["Payment"][0]:,.2f}')
    st.dataframe(amortization_df)

    # Downloadable CSV
    csv = amortization_df.to_csv(index=False)
    st.download_button(label="Download Amortization Table as CSV", data=csv, file_name='amortization_schedule.csv', mime='text/csv')


Next, we obtain the password for our StreamLit server we are about to launch.

In [ ]:
!curl https://loca.lt/mytunnelpassword

We launch the StreamLit server and obtain its URL. You will need the above password when you access the URL it gives you.

In [ ]:
!streamlit run app.py server1 &>/content/logs.txt &
!npx --yes localtunnel --port 8501

## Cartoon Image

This code creates a Streamlit application that transforms uploaded images into cartoon-like versions using a multimodal Large Language Model (LLM) and image generation tools. The application allows users to upload an image, processes it with an AI model, and generates a cartoonified version, which can then be downloaded.

The core functionality is encapsulated in the modify_image function. This function starts by initializing the GPT model, specifically "gpt-4o-mini." When an image is uploaded, it's converted into a base64-encoded string to facilitate communication with the LLM. A message is created combining a text prompt and the base64-encoded image, which is then sent to the GPT model to generate a modified prompt. The response from the model provides a prompt intended to render the image in a cartoon style.

After obtaining this cartoon prompt, the function uses OpenAI's DALL-E model ("dall-e-3") to generate a new image based on the prompt. The image generation process returns a URL, which is then fetched and converted into a PIL Image object for further processing.

The Streamlit app interface begins with a title: "Cartoonify Image with Multimodal LLM." It provides an image uploader using the st.file_uploader method, allowing users to upload images in JPEG or PNG format. Once an image is uploaded, it is displayed using Streamlit's st.image method.

The modify_image function is then called to process the uploaded image and transform it into a cartoon style. While this process is happening, a message ("Generating cartoon version...") is shown to indicate ongoing activity. The cartoonified image is then displayed in the app.

Lastly, the app offers the option to download the cartoonified image. This is achieved by saving the modified image into a buffer and using Streamlit's st.download_button to allow the user to download it in JPEG format. This provides a seamless way for users to not only view the transformed image but also save it locally.

In [ ]:
%%writefile app.py

import streamlit as st
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
import base64
import httpx
from openai import OpenAI
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import requests

def modify_image(image, prompt):
    # Initialize the GPT model
    model = ChatOpenAI(model="gpt-4o-mini")

    # Convert the uploaded image to base64
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    image_data = base64.b64encode(buffered.getvalue()).decode("utf-8")

    # Create a message with both text and the image
    message = HumanMessage(
        content=[
            {"type": "text", "text": prompt},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}},
        ],
    )

    # Get response with a modified prompt from GPT
    response = model.invoke([message])
    cartoon_prompt = response.content

    # Initialize the DALL-E model to generate the image
    client = OpenAI()

    # Generate the image based on the GPT-generated cartoon prompt
    response = client.images.generate(
        model="dall-e-3",
        prompt=cartoon_prompt,
        size="1024x1024",
        quality="standard",
        n=1,
    )

    # Get the image URL from DALL-E
    image_url = response.data[0].url

    # Fetch the generated image
    response2 = requests.get(image_url)
    img = Image.open(BytesIO(response2.content))

    return img

# Streamlit app
st.title("Cartoonify Image with Multimodal LLM")

# Image upload
uploaded_image = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])

if uploaded_image is not None:
    # Open the image using PIL
    image = Image.open(uploaded_image)

    # Display the original image
    st.image(image, caption='Uploaded Image', use_column_width=True)

    # Modify the image to look like a cartoon
    st.write("Generating cartoon version...")
    cartoon_img = modify_image(image, "Output a prompt that would render this image as a cartoon.")

    # Display the cartoonified image
    st.image(cartoon_img, caption='Cartoonified Image', use_column_width=True)

    # Provide an option to download the cartoonified image
    buffered = BytesIO()
    cartoon_img.save(buffered, format="JPEG")
    st.download_button(
        label="Download Cartoon Image",
        data=buffered.getvalue(),
        file_name="cartoon_image.jpg",
        mime="image/jpeg"
    )


Next, we obtain the password for our StreamLit server we are about to launch.

In [ ]:
!curl https://loca.lt/mytunnelpassword

We launch the StreamLit server and obtain its URL. You will need the above password when you access the URL it gives you.

In [ ]:
!streamlit run app.py server1 &>/content/logs.txt &
!npx --yes localtunnel --port 8501